<a href="https://colab.research.google.com/github/POmegrenadier9090/keyboard_arr_optimizer/blob/main/Copy_of_Keyboard_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sys import setswitchinterval
import numpy as np
import pandas as pd
import math

alphabet_index = {
    'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6,
    'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12,
    'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18,
    't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25
}
key_cost = np.array(
        [
            [22,18,10,9,15,15,9,10,18,22],
            [11,7, 1, 1,7, 7, 1,1, 7, 11],
            [22,18,10,9,15,15,9,10,18,22]
    ]
        )

#calculates the coordinates difining "z" as the origin
key_position = np.array(
    [#abcdefghijklmf
     [0,1],[4,0],[2,0],[2,1],[2,2],[3,1],[4,1],[5,1],[7,2],[6,1],[7,1],[8,1],[6,0],
     #nopqrstuvwxyz
     [5,0],[8,2],[9,2],[0,2],[3,2],[1,1],[4,2],[6,2],[3,0],[1,2],[1,0],[5,2],[0,0]
     ]
)

# ひらがな配列をローマ字に変換する関数
def convert_to_romaji(hiragana_string):
  hiragana_to_romaji = {
    "あ": "a", "い": "i", "う": "u", "え": "e", "お": "o",
    "か": "ka", "き": "ki", "く": "ku", "け": "ke", "こ": "ko",
    "さ": "sa", "し": "si", "す": "su", "せ": "se", "そ": "so",
    "た": "ta", "ち": "ti", "つ": "tu", "て": "te", "と": "to",
    "な": "na", "に": "ni", "ぬ": "nu", "ね": "ne", "の": "no",
    "は": "ha", "ひ": "hi", "ふ": "hu", "へ": "he", "ほ": "ho",
    "ま": "ma", "み": "mi", "む": "mu", "め": "me", "も": "mo",
    "や": "ya",             "ゆ": "yu",             "よ": "yo",
    "ら": "ra", "り": "ri", "る": "ru", "れ": "re", "ろ": "ro",
    "わ": "wa",                                     "を": "wo", "ん": "nn",
    "が": "ga", "ぎ": "gi", "ぐ": "gu", "げ": "ge", "ご": "go",
    "ざ": "za", "じ": "ji", "ず": "zu", "ぜ": "ze", "ぞ": "zo",
    "だ": "da", "ぢ": "ji", "づ": "zu", "で": "de", "ど": "do",
    "ば": "ba", "び": "bi", "ぶ": "bu", "べ": "be", "ぼ": "bo",
    "ぱ": "pa", "ぴ": "pi", "ぷ": "pu", "ぺ": "pe", "ぽ": "po",
    "ゔ": "vu",
    "ぁ": "la", "ぃ": "li", "ぅ": "lu", "ぇ":"le", "ぉ":"lo"
  }
  lower_vowel = {"ぁ": "a", "ぃ": "i", "ぅ": "u", "ぇ":"e", "ぉ":"o"}
  y_column_vowel = {"ゃ":"a", "ゅ": "u", "ょ": "o"}

  def two_letter_input(cur_char, next_char,flg):
    if cur_char in ("ふ", "ゔ") and next_char in lower_vowel:
      con = hiragana_to_romaji.get(cur_char,"")[0]
      vowel = lower_vowel.get(next_char,"")
      temp = con + vowel
      flg += 1
    elif next_char in ("ゃ","ゅ","ょ"):
      con = hiragana_to_romaji.get(cur_char,"")[0]
      vowel = y_column_vowel.get(next_char,"")
      temp = con + "y" + vowel
      flg += 1
    else:
      temp=""
      flg=0
    return temp, flg

  def lower_case_tu(cur_char, next_char, next_next_char,flg):
    con = hiragana_to_romaji.get(next_char,"")[0]
    if next_next_char in lower_vowel or next_next_char in y_column_vowel:
      temp, flg = two_letter_input(next_char, next_next_char, flg)
      temp = con + temp
      flg += 1
    else:
      vowel = hiragana_to_romaji.get(next_char,"")[1]
      temp = con *2 + vowel
      flg += 1
    return temp, flg

  romaji_result = ""
  i = 0
  r=-1
  while i < len(hiragana_string):
    if i==r:
      print("@@@@@")
      break
    r= i

    flg = 0
    cur_char = hiragana_string[i]
    if i < len(hiragana_string)-1:
      next_char = hiragana_string[i+1]
    else:
      next_char =""
    if i < len(hiragana_string)-2:
      next_next_char = hiragana_string[i+2]
    else:
      next_next_char =""
    # 分岐スタート
    #小さいっの場合
    if cur_char == "っ":
        # 次の文字が存在する場合、その文字の先頭を重ねる
      if next_char:
        temp, flg = lower_case_tu(cur_char, next_char,next_next_char,flg)
        romaji_result += temp
        flg += 1
    #2-letterの場合
    elif next_char in lower_vowel or next_char in y_column_vowel:
      temp, flg = two_letter_input(cur_char, next_char,flg)
      if not temp:
        con =hiragana_to_romaji.get(cur_char)
        lower = hiragana_to_romaji.get(next_char)
        temp = con+ lower
        flg+=1
      romaji_result += temp
      flg += 1
    #1対1変換
    else:
      romaji_result += hiragana_to_romaji.get(cur_char,"")
      flg += 1
    i += flg
  return romaji_result

def cost(prev_key, key):
  #連続して同じ手を用いるか？ 左手: 0 、右手: 1
  cost_additional = 0
  prev_hand = 0 if prev_key[0] <= 4 else 1
  cur_hand = 0 if key[0] <= 4 else 1
  #全く同じなら追加コストなし
  if prev_hand == cur_hand and not np.all(prev_key == key):
    cost_additional += 10
      #同じ側の同じ指を連続して使うならさらにコスト増加
    if (prev_key[0] in (3,4) and key[0] in (3,4)) or (prev_key[0] in (5,6) and key[0] in (5,6)):
      cost_additional += 6
    if prev_key[0] == key[0]:
      cost_additional += 6
  return key_cost[key[1],key[0]] + cost_additional

#key: 入力したキーの座標
#to be done: 上下動作の非対称性、2個飛ばしが起きた際のコスト増加、右手はEnterを使うのでコストが大きめ
def calc_avg_cost(input_list):
  total_cost = 0
  prev_key =key_position[alphabet_index.get(input_list[0])]
  for letter in input_list[1:]:
    key = key_position[alphabet_index.get(letter)]
    total_cost += cost(prev_key, key)
  return total_cost/len(input_list)

def calc_transition_N(text):
  transition_N = np.zeros((26,26))
  prev_index = alphabet_index.get(text[0])
  for letter in text[1:]:
    index = alphabet_index.get(letter)
    transition_N[prev_index, index] += 1
    prev_index = index
  return transition_N

def calc_cost_matrix(transition_N):
  N = np.sum(transition_N)
  cost_matrix = np.zeros((26,26))
  for i in range(cost_matrix.shape[0]):
    prev_key = key_position[i]
    for j in range(cost_matrix.shape[1]):
      key = key_position[j]
      cost_matrix[i,j] = math.pow(cost(prev_key, key), 2) * math.log(transition_N[i,j],7) / N if transition_N[i,j] !=0 else -0.0001
  return cost_matri

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
%cd "/content/drive/My Drive/Project"
%ls
with open('sample_text_営業意思決定支援.txt', 'r', encoding='Shift_JIS',errors = "replace") as file:

    sentence = file.read()
print(sentence)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Project
gooLab_hiragana.txt  Keyboard_Learning.ipynb  sample_text_営業意思決定支援.txt
深層強化学習による営業活動意思決定支援システム
山下　航平

1. 概要
　営業活動の意思決定は，明確なルールが存在しない属人性の高い業務である．近年の社会の変化に合わせ，このような非定型業務においても迅速な意思決定が求められるようになり，機械学習によるレコメンドや予測が実践されている．しかし，業務の複雑性のため，営業活動全般を対象にした研究の前例はない．
　そこで本研究では，深層強化学習を用いて，営業アクティビティをレコメンドするモデルを構築する．深層強化学習を用いることで，営業活動の特徴を多段的に抽出し，顧客状態に応じた，最適な行動の学習が可能となる．

2. 関連手法
2.1 強化学習


　強化学習では，意思決定主体である「エージェント」が「環境」から認識する「状態」をもとに「行動」を意思決定し，「報酬」を得る．アルゴリズムを繰り返すことで，ある「状態」に対し「報酬」を最大化する方策を学習する．強化学習と営業活動との対応関係を図１に示す．
　

図1．強化学習と営業活動の対応関係
　営業活動の意志決定構造に強化学習を適用すると，営業担当者（エージェント）が顧客（環境）の検討状況（状態）に応じて最も受注（報酬）につながりやすい営業活動（行動）を選択する，という対応関係になる．

2.2 強化学習：モンテカルロ木探索（MCTS）


　本研究で用いられたMCTS（Monte Carlo Tree Search）は，決定木を用いて期待報酬を最大化する方策を探索する，強化学習の手法である．決定木におけるエッジの分岐は「異なる行動」，ノードは「状態」を表す．ルートノードからの道筋をたどることで推移的な行動選択と，ある時点での選択の価値を評価する．営業活動を例として，MCTSのプロセスを図２に示す． 
　
　
　図

In [ ]:
import requests
import chardet

api_key = 'c7bb63f0be4f3f6f9aa980e67151d12e06dfc6f81937d2e9a244bc8e9d1f6f53'
#sentence = '皆さんの誤解を解く目的で解説する。あっちゃーヴぁー'
url = "https://labs.goo.ne.jp/api/hiragana"
params = {
    "app_id": api_key,
    "sentence": sentence,
    "output_type": "hiragana"
}
try:
  response = requests.post(url, json = params)
  if response.status_code == 200:
    sentence_hiragana = response.json()
  else:
    print(f"error: {response.status_code}, message: {response.text}")
except requests.exceptions.RequestException as e:
    # リクエスト中にエラーが発生した場合
    print("An error has occurred", str(e))


In [ ]:
print(sentence_hiragana["converted"])
print(convert_to_romaji(sentence_hiragana["converted"]))

しんそうきょうかがくしゅうによる えいぎょうかつどういしけっていしえんしすてむ  やました こうへい  いち. がいよう  えいぎょうかつどうの いしけっていは， めいかくな るーるが そんざいしない ぞくじんせいの たかい ぎょうむである． きんねんの しゃかいの へんかに あわせ， このような ひていけいぎょうむにおいても じんそくな いしけっていが もとめられるようになり， きかいがくしゅうによる れこめんどや よそくが じっせんされている． しかし， ぎょうむの ふくざつせいの ため， えいぎょうかつどうぜんぱんを たいしょうに した けんきゅうの ぜんれいは ない． そこで ほんけんきゅうでは， しんそうきょうかがくしゅうを もちいて， えいぎょうあくてぃびてぃを れこめんどする もでるを こうちくする． しんそうきょうかがくしゅうを もちいる ことで， えいぎょうかつどうの とくちょうを ただんてきに ちゅうしゅつし， こきゃくじょうたいに おうじた， さいてきな こうどうの がくしゅうが かのうと なる． に.  かんれんしゅほう  にてんいち  きょうかがくしゅう  きょうかがくしゅうでは， いしけっていしゅたいである 「えーじぇんと」が 「かんきょう」から にんしきする 「じょうたい」をもとに 「こうどう」を いしけっていし， 「ほうしゅう」を える． あるごりずむを くりかえす ことで， ある 「じょうたい」に たいし 「ほうしゅう」を さいだいかする ほうさくを がくしゅうする． きょうかがくしゅうと えいぎょうかつどうとの たいおうかんけいを ずいちに しめす． ずいち． きょうかがくしゅうと えいぎょうかつどうの たいおうかんけい  えいぎょうかつどうの いしけっていこうぞうに きょうかがくしゅうを てきようすると， えいぎょうたんとうしゃ（えーじぇんと）が こきゃく （かんきょう）の けんとうじょうきょう （じょうたい）に おうじて もっとも じゅちゅう （ほうしゅう）に つながりやすい えいぎょうかつどう （こうどう）を せんたくする，と いう たいおうかんけいに なる． にてんに   きょうかがくしゅう： もんてかるろきたんさく（えむしーてぃーえす）  ほんけんきゅうで もちいられた えむしーてぃーえす（もんて しーえーあーるえるおー

In [ ]:
input_list = list(convert_to_romaji(sentence_hiragana["converted"]))
input = np.array(input_list)

In [ ]:
pd.set_option('display.max_columns', None)
text_test = convert_to_romaji(sentence_hiragana["converted"])
transition_N = calc_transition_N(text_test)
df = pd.DataFrame(transition_N,alphabet_index.keys(), alphabet_index.keys())
print(calc_avg_cost(input_list))
display(df)

14.24826862930003


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
a,21.0,13.0,0.0,15.0,35.0,0.0,59.0,31.0,279.0,18.0,320.0,0.0,40.0,355.0,5.0,19.0,0.0,195.0,107.0,173.0,9.0,0.0,26.0,0.0,11.0,10.0
b,19.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,1.0,0.0
c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d,28.0,0.0,0.0,0.0,212.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
e,34.0,3.0,0.0,4.0,28.0,0.0,12.0,49.0,248.0,24.0,101.0,195.0,86.0,250.0,6.0,14.0,0.0,204.0,167.0,145.0,0.0,0.0,4.0,0.0,2.0,3.0
f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g,166.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,65.0,0.0
h,144.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,91.0,0.0,0.0,0.0,0.0,0.0,38.0,0.0,0.0,0.0,25.0,0.0
i,75.0,41.0,0.0,52.0,123.0,0.0,108.0,66.0,73.0,30.0,210.0,17.0,43.0,253.0,85.0,80.0,0.0,71.0,229.0,460.0,8.0,0.0,60.0,0.0,83.0,61.0
j,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108.0,0.0


In [ ]:
#コスト寄与率を計算
cost_matrix = calc_cost_matrix(transition_N)
df_x = pd.DataFrame(cost_matrix,alphabet_index.keys(), alphabet_index.keys())
# 最大値とその位置を取得
max_value = df_x.values.max()
max_value_index = df_x.stack().idxmax()

print(f"最大値: {max_value}, 位置: {max_value_index}")

display(df_x)
#⇒uとnの位置を変更したい。
#まず、どのキーが次に続くキーがどちら側に来やすいかを求めて初期の配列を構成すべきか？
#目指す方向としては、同じ側で連続して打つ頻度を減らすこと（こちらは上記の初期化でうまくいくかもしれない、頻度をそのまま使うことはできないので、割合で計算する必要がある。）
#打つ頻度の大きなキーはできるだけホームポジションに位置させること（初期化した中で頻度の大きいものをホームポジションに配置する。さらに、同じ手の中で連続）

#コスト行列内から最も値の大きな(選択方法は要検討)要素targetを選ぶ
#コスト行列のcolumn和を取りその中で最も値(合計寄与率)が小さな文字replacedを取得:N=0の値は-0.001ぐらいが妥当か,この際、targetのrowの文字と異なる側から選択する(逆サイドの文字全てについてシミュレーションを行い、コスト関数を最も減少させる組み合わせを求めてもいいかも)
#targetのcolumnの文字とreplacedの位置を交換する(key_position)の更新
#新しくコスト行列を計算
#εGREEDYを使うのもありかも、(確率εでランダムに位置変更)次第にεの値を縮小していくー同じキーの交換が続いてしまうかも

最大値: 0.1374471604771652, 位置: ('u', 'n')


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
a,0.009399,0.040901,-1.0,0.008360,0.036284,-1.0,0.030066,0.004293,0.014367,0.000074,0.000147,-1.000000,0.007623,0.033709,0.013304,0.036360,-1.0,0.048567,0.034455,0.082175,0.004541,-1.0,0.065171,-1.0,0.013765,0.084832
b,0.033130,-1.000000,-1.0,-1.000000,0.022424,-1.0,-1.000000,-1.000000,0.009712,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.014812,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,0.007820,-1.0,-1.000000,-1.0,0.000000,-1.000000
c,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000
d,0.037493,-1.000000,-1.0,-1.000000,0.092387,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.039780,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000
e,0.039677,0.017519,-1.0,0.010222,0.008502,-1.0,0.018322,0.004865,0.014067,0.000081,0.000118,0.006592,0.009205,0.031696,0.014812,0.032589,-1.0,0.048982,0.037737,0.079360,-1.000000,-1.0,0.027730,-1.0,0.003979,0.028702
f,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.000000
g,0.057518,-1.000000,-1.0,-1.000000,0.032434,-1.0,-1.000000,-1.000000,0.006733,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.026933,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,0.005596,-1.0,-1.000000,-1.0,0.023963,-1.000000
h,0.015343,-1.000000,-1.0,-1.000000,0.007374,-1.0,-1.000000,-1.000000,0.032434,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.090230,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,0.058005,-1.0,-1.000000,-1.0,0.112430,-1.000000
i,0.013329,0.021318,-1.0,0.000101,0.012278,-1.0,0.005853,0.030892,0.010947,0.010500,0.039427,0.020891,0.034642,0.088236,0.088865,0.114485,-1.0,0.008809,0.006793,0.035197,0.019153,-1.0,0.033846,-1.0,0.070463,0.050764
j,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,0.045340,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,0.114800,-1.000000
